In [12]:
# PIL 라이브러리 import (이미지 크기 변경을 위해)
from PIL import Image
import os, glob

import os

def resize_images(img_path):
    images=glob.glob(img_path + "/*.jpg")
    
    print(len(images), "images to be resized.")
    
    # 파일마다 모두 28x28 사이즈로 바꾸어 저장
    target_size = (28, 28)
    for img in images:
        old_img=Image.open(img)
        new_img=old_img.resize(target_size, Image.ANTIALIAS)
        new_img.save(img, "JPEG")

    print(len(images), "images resized")
    
# 가위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/scissor"
resize_images(image_dir_path)

# rock

image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/rock"
resize_images(image_dir_path)

# paper
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/paper"
resize_images(image_dir_path)


516 images to be resized.
516 images resized
429 images to be resized.
429 images resized
368 images to be resized.
368 images resized


In [13]:
# 가위, 바위, 보 데이터를 읽을 수 있는 load_data 함수 만들기
# 함수 입력으로 이미지 폴더 위치 받음
# 3개의 클래스로 라벨링 (가위 0, 바위 1, 보2)
import numpy as np

# 가위바위보 이미지 개수 총합에 주의하세요.
def load_data(img_path, number_of_data=1313):

    img_size=28
    color=3

    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를
    #담을 행렬(matrix) 영역을 생성
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data, img_size, img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)
    
    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file), dtype=np.int32)
        
        # 데이터 영역에 이미지 행렬 복사
        imgs[idx,:,:,:] = img
        labels[idx]=0 # 가위
        idx=idx+1
        
    
    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file), dtype=np.int32)
        imgs[idx,:,:,:] = img
        labels[idx]=1 # 바위
        idx=idx+1
    
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file), dtype=np.int32)
        imgs[idx,:,:,:] = img
        labels[idx]=2 # 보
        idx=idx+1
    
    print("데이터의 이미지 개수는", idx,"입니다.")
    return imgs, labels

In [14]:
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper"
(x_train, y_train)=load_data(image_dir_path)

# 입력 정규화 (0~1)
x_train_norm=x_train/255.0

print("x_train shape:{}".format(x_train.shape))
print("y_train shape:{}".format(y_train.shape))



데이터의 이미지 개수는 1313 입니다.
x_train shape:(1313, 28, 28, 3)
y_train shape:(1313,)


In [15]:
# test 이미지 resize

image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test/scissor"
resize_images(image_dir_path)

image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test/rock"
resize_images(image_dir_path)

image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test/paper"
resize_images(image_dir_path)

image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test"
(x_test, y_test)=load_data(image_dir_path, 300)
x_test_norm = x_test/255.0   # 입력은 0~1 사이의 값으로 정규화

print("x_test shape: {}".format(x_test.shape))
print("y_test shape: {}".format(y_test.shape))

100 images to be resized.
100 images resized
100 images to be resized.
100 images resized
100 images to be resized.
100 images resized
데이터의 이미지 개수는 300 입니다.
x_test shape: (300, 28, 28, 3)
y_test shape: (300,)


In [36]:
import tensorflow as tf
from tensorflow import keras
import numpy as np


n_channel_1 = 32
n_channel_2 = 32
n_channel_3 = 64
n_dense = 32
n_train_epoch = 20


model = keras.models.Sequential()
# 흑백 1, 컬러 3 
# 커널개수, 커널 사이즈 
# 인풋 세입은 처음에만 넣음
model.add(keras.layers.Conv2D(n_channel_1, (3, 3),padding='valid', activation='relu', input_shape=(28,28,3)))
model.add(keras.layers.MaxPool2D(2,2))

model.add(keras.layers.Conv2D(n_channel_2, (3,3),padding='valid', activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))

model.add(keras.layers.Conv2D(n_channel_3, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))


# 1차원 텐서로 변환
model.add(keras.layers.Flatten()) # 1차원으로 풀어줌
model.add(keras.layers.Dense(n_dense, activation='relu'))
model.add(keras.layers.Dropout(0.5))

# Dense 뒤에 가위, 바위, 보 -> 3개
model.add(keras.layers.Dense(3, activation='softmax'))

x_train_reshaped = x_train_norm.reshape(-1, 28, 28, 3)


model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])


# 모델 훈련
model.fit(x_train_reshaped, y_train, epochs= n_train_epoch, batch_size=1)


image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test"
(x_test, y_test)=load_data(image_dir_path, 300)
x_test_norm = x_test/255.0   # 입력은 0~1 사이의 값으로 정규화

x_test_reshaped = x_test_norm.reshape(-1, 28, 28, 3)

# 모델 시험
test_loss, test_accuracy = model.evaluate(x_test_reshaped,y_test, verbose = 2)



print("test_loss:{}".format(test_loss))
print("test_accuracy:{}".format(test_accuracy))

Epoch 1/20
1313/1313 [==============================] - 4s 3ms/step - loss: 1.0860 - accuracy: 0.3794
Epoch 2/20
1313/1313 [==============================] - 4s 3ms/step - loss: 0.7458 - accuracy: 0.7073
Epoch 3/20
1313/1313 [==============================] - 3s 3ms/step - loss: 0.2948 - accuracy: 0.8693
Epoch 4/20
1313/1313 [==============================] - 3s 3ms/step - loss: 0.1892 - accuracy: 0.9348
Epoch 5/20
1313/1313 [==============================] - 3s 3ms/step - loss: 0.2498 - accuracy: 0.9102
Epoch 6/20
1313/1313 [==============================] - 3s 3ms/step - loss: 0.1108 - accuracy: 0.9715
Epoch 7/20
1313/1313 [==============================] - 3s 3ms/step - loss: 0.1220 - accuracy: 0.9611
Epoch 8/20
1313/1313 [==============================] - 3s 3ms/step - loss: 0.1080 - accuracy: 0.9639
Epoch 9/20
1313/1313 [==============================] - 3s 3ms/step - loss: 0.0608 - accuracy: 0.9765
Epoch 10/20
1313/1313 [==============================] - 3s 3ms/step - loss: 0.072

In [ ]:
# accuracy 가 너무 낮게 나와 하이퍼 파라미터 조정
# accuracy가 너무 낮게 나와서 train 데이터 추가하여 다시 시도
# 진전이 없어서 layer 추가, dropout (오버피팅 방지)